# One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming with Material and Emission Considerations Example

## Example 3

This is a continuation of 'One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming Example' [Example 2]. 

We will be adding the following considerations: 

1. material for the establishment of operations 
2. global warming potential induced as a sum of our decisions and choices 

## From Example 2

In [3]:
from energia import *

m = Model('example3')
m.q = Period()
m.y = 4 * m.q


In [4]:
m.usd = Currency()
m.declare(Resource, ['power', 'wind', 'solar'])
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49

m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

--- General Resource Balance for solar in (l, q): initializing constraint, adding consume
--- General Resource Balance for wind in (l, y): initializing constraint, adding consume
--- General Resource Balance for power in (l, q): initializing constraint, adding release
--- Creating map to (wf, l, y). Mapping operate: from (wf, l, q) to (wf, l, y)
--- Creating map to (pv, l, y). Mapping operate: from (pv, l, q) to (pv, l, y)
--- General Resource Balance for power.lii in (l, y): initializing constraint, adding inventory


## Indicator Stream 

Indicators scale the impact of some decision or flow and project it only a common dimension allowing the calculation of overall impact as a single metric value

In this case, we consider global warming potential (GWP).

In [5]:
m.gwp = Environ()

Now say that we want to consider the impact of setting up operations (specific to the act of construction)

These can be provided as calculations 

In [6]:
m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that impact streams are mapped across scales but not balanced, unless a negative (relatively) impact is also assessed! In the case of emission, this would be abatement.

## Material Stream

Note that all the objects found in energia.components.commodity.misc are still resources (Land, Material, etc.)

This will allow for the modeling of an expansive type of processes.

Examples:

1. land clearance to transform agricultural land into industrial, or land remediation 
2. resource intense material recycling 
3. some resource flowing being treated as emission flows 

The possibilities are vast... The advanced modeler may prefer to use Resource() for everything [see Resource Task Network (RTN) Framework]

In the following example, we consider a limit to cement consumption, as also an expense and gwp impact associated with it

In [7]:
m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17
m.cement.consume[m.gwp.emit] == 0.9

--- General Resource Balance for cement in (l, y): initializing constraint, adding consume


Next, we provide details of the use of cement across all operations 

In [8]:
m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

--- General Resource Balance for cement in (l, y): adding use
--- General Resource Balance for cement in (l, y): adding use
--- General Resource Balance for cement in (l, y): adding use


Material use is summed up across all operations, and adheres to an upper bound in terms of consumption. 

In [9]:
m.cement.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

GWP impact is now the sum of impact from material use as well as constuction 

In [10]:
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Locate Operations

In [11]:
m.network.operations(m.wf, m.pv, m.lii)

--- General Resource Balance for power in (l, q): adding produce
--- General Resource Balance for wind in (l, y): adding expend
--- General Resource Balance for power in (l, q): adding produce
--- General Resource Balance for solar in (l, q): adding expend
--- Assuming  power.lii inventory capacity is unbounded in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, q)
--- Creating map to (power.lii, l, y). Mapping inventory: from (power.lii, l, q) to (power.lii, l, y)
--- General Resource Balance for power.lii in (l, q): initializing constraint, adding inventory
--- General Resource Balance for power.lii in (l, q): adding produce
--- Creating map to (lii.charge, l, y). Mapping operate: from (lii.charge, l, q) to (lii.charge, l, y)
--- General Resource Balance for power in (l, q): adding expend
--- General Resource Balance for power in (l, q): adding produce
--- Creating map to (lii.discharge, l, y). Mapping operate: from (lii.discharge, l, q) to (lii.discha

## Optimization 

Let us minimize GWP this time. Note that m.usd.spend.opt() can still be used!

In [12]:
m.gwp.emit.opt()

--- Creating map to (gwp, l, y). Mapping emit: from (gwp, l, y, capacity, wf) to (gwp, l, y)
--- Mapping emit: from (gwp, l, y, capacity, pv) to (gwp, l, y)
--- Mapping emit: from (gwp, l, y, invcapacity, power.lii) to (gwp, l, y)
--- Mapping emit: from (gwp, l, y, consume, cement) to (gwp, l, y)
--- Generating example3.mps
--- Creating gurobi model for example3
Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-16
Read MPS format model from file example3.mps
Reading time = 0.05 seconds
EXAMPLE3: 95 rows, 87 columns, 217 nonzeros
--- Optimizing example3 using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 95 rows, 87 columns and 217 nonzeros
Model fingerprint: 0x39a95d48
Variable types: 84 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix 


Root relaxation: objective 4.752148e+05, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    475214.81481 475214.815  0.00%     -    0s

Explored 1 nodes (4 simplex iterations) in 0.15 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 475215 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.752148148148e+05, best bound 4.752148148148e+05, gap 0.0000%
--- Solution found. Use .sol() to display it
--- Creating Solution object, check.solution


In [13]:
m.power.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [14]:
m.sol(False)

# Solution for example3

## Objective

<IPython.core.display.Math object>

## Variables

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>